# Quick analysis of sales data with directory verification

Description:

You received two CSV files: one containing sales data from the last quarter, and another with the current product catalog and their listed prices.
Your task is to quickly analyze this data and answer key questions from the management, taking into account potential discrepancies between the sales report and the catalog.

## Task 1 

Merge the sales data with the product catalog. How many unique transactions from the file raport_kwartalny_sprzedaz.csv do not have a corresponding entry in katalog_produktow.csv based on the Product ID column?

In [2]:
import pandas as pd

sprzedaz_id = '1wl5QaazaxOkToVbVZwEcaMuvcZjBeM1a'
katalog_id = '1wTFPyNfaQOg3scLpG9z9h7zomIwfZi8X'

url_sprzedaz = f'https://drive.google.com/uc?id={sprzedaz_id}'
url_katalog = f'https://drive.google.com/uc?id={katalog_id}'

df = pd.read_csv(url_sprzedaz)
df1 = pd.read_csv(url_katalog)

merged_df = pd.merge(
    df,
    df1,
    how='left',
    left_on='ID_Produktu_Sprzedaz',
    right_on='ID_Produktu_Katalog',
    indicator=True
)

unmatched_transactions = merged_df[merged_df['_merge'] == 'left_only']

import numpy as np

print(len(np.unique(unmatched_transactions['ID_Produktu_Sprzedaz'])))

146


### What am I doing?

-I import the necessary libraries and the files (data sources) I’ll be working with.

-Then I read the .csv files containing the sales data and the product catalog.

-I merge both tables using pd.merge() – I specify:

    -the names of the tables to be joined,

    -the type of join (left join, to keep all rows from the sales data and insert NaN if no match is found in the catalog),

    -the common column used for the join (e.g. Product ID),

    -and I add an indicator column to later filter unmatched entries.

-I search for unmatched transactions (i.e. ones that did not find a match in the second table) using the _merge indicator, filtering only rows marked as "left_only" – meaning they exist in the sales data but not in the catalog.

-I use the NumPy library to find the unique values, ensuring that I’m counting distinct unmatched transactions.

## Task 2

What is the total sales value in the entire report (Wartosc_Sprzedazy from the file raport_kwartalny_sprzedaz.csv), considering only those transactions for which a matching product was found in the catalog?

Present the result rounded to two decimal places.

In [13]:
print(df.columns)

Index(['Data_Transakcji', 'ID_Produktu_Sprzedaz', 'Region', 'Ilosc_Sprzedana',
       'Cena_Jednostkowa_Transakcji', 'Wartosc_Sprzedazy'],
      dtype='object')


In [12]:
print(df1.columns)

Index(['ID_Produktu_Katalog', 'Nazwa_Produktu', 'Cena_Katalogowa',
       'Kategoria_Produktu'],
      dtype='object')


In [14]:
df_merged1 = pd.merge(
    df, 
    df1, 
    left_on='ID_Produktu_Sprzedaz', 
    right_on='ID_Produktu_Katalog', 
    how='inner')
total_sales = df_merged1['Wartosc_Sprzedazy'].sum()
print(round(total_sales, 2))

31336539.21


### What am I doing?

-I merge both tables just like in the previous task, but this time I set the how parameter to "inner" so that only the rows with matching entries in both the sales report and the product catalog are kept (ensuring no missing values).

-Thanks to this, each row has a matching product with a valid value, and I can safely calculate the total sales value using .sum() on the Wartosc_Sprzedazy column.

-Finally, I round the result to two decimal places.

## Task 3a

Which "Nazwa_Produktu" (from the catalog) has the largest percentage difference between the average Cena_Jednostkowa_Transakcji (from the sales report) and the Cena_Katalogowa (from the catalog)?

Return the product name only.

In [23]:
df_merged3 = pd.merge(
    df,
    df1,
    left_on='ID_Produktu_Sprzedaz',
    right_on='ID_Produktu_Katalog',
    how='inner')

srednie_ceny = df_merged3.groupby('Nazwa_Produktu')['Cena_Jednostkowa_Transakcji'].mean().reset_index()

srednie_ceny.rename(columns={'Cena_Jednostkowa_Transakcji': 'Srednia_Cena_Transakcji'}, inplace=True)

srednie_ceny = pd.merge(
    srednie_ceny,
    df1[['Nazwa_Produktu', 'Cena_Katalogowa']],
    on='Nazwa_Produktu',
    how='left')

srednie_ceny['Procentowa_Roznica'] = (
    (srednie_ceny['Srednia_Cena_Transakcji'] - srednie_ceny['Cena_Katalogowa'])
    / srednie_ceny['Cena_Katalogowa']
) * 100

max_diff_row = srednie_ceny.iloc[srednie_ceny['Procentowa_Roznica'].abs().idxmax()]

print(max_diff_row['Nazwa_Produktu'])

Miara Przyczyna


### What am I doing?

-I merge the two tables in such a way that only rows with matching product IDs on both sides are included.

-I calculate the average unit price per product by grouping the sales data by product name and computing the mean of Cena_Jednostkowa_Transakcji. Then I reset the index to turn the product name back into a regular column, so I can work with it more easily.

-I rename the Cena_Jednostkowa_Transakcji column to make the data clearer and avoid confusion about what the values represent.

-I merge the resulting average prices with the catalog prices from the second table, joining on the product name, so I can compare them directly.

-I calculate the percentage difference between the average transaction price and the catalog price, and I add this as a new column to the table.

-I find the product with the largest percentage difference using .idxmax() to get the row index with the maximum absolute difference, and then .iloc to retrieve the product's name from that row.

-I use .abs() in the calculation to make sure the result reflects the biggest deviation, regardless of whether the difference is positive or negative.

## Task 3b

What is the percentage difference from Task 3a?

Provide the percentage difference including the sign:

    "+" if the average unit transaction price is higher than the catalog price,

    "-" if it is lower.

In [27]:
roznica_procent = max_diff_row['Procentowa_Roznica']

print(roznica_procent)

print(round(roznica_procent, 2))

-29.999534710589987
-30.0


### What am I doing?

-Based on the previously calculated maximum percentage difference, I retrieve that value,

-Then round it to make it more readable and easier to interpret.

This value already includes the sign (positive or negative), which indicates whether the average transaction price is higher or lower than the catalog price.

## Task 4

How many rows in the sales report contain a mismatch between Wartosc_Sprzedazy and the product of Ilosc_Sprzedana * Cena_Jednostkowa_Transakcji?

In [28]:
df['Iloczyn'] = df['Ilosc_Sprzedana'] * df['Cena_Jednostkowa_Transakcji']

niezgodne = df[~(df['Wartosc_Sprzedazy'] == df['Iloczyn'])]

print(len(niezgodne))

1239


### What am I doing?

-I calculate the product specified in the task: Ilosc_Sprzedana * Cena_Jednostkowa_Transakcji.

-Then, I compare this calculated value with the Wartosc_Sprzedazy column to check for mismatches.

-I use the ~ operator to invert the match condition, selecting only the rows where the values do not match.

-Finally, I count the number of mismatched rows, which gives me the total number of discrepancies in the report.

In [30]:
niezgodne1 = df[~df['Wartosc_Sprzedazy'].round(2).eq(df['Iloczyn'].round(2))]

print(len(niezgodne1))

373


### What am I doing?

I round both the calculated product (Ilosc_Sprzedana * Cena_Jednostkowa_Transakcji) and the Wartosc_Sprzedazy value to 2 decimal places.

This helps to avoid small discrepancies caused by floating-point precision, rounding errors, or technical formatting issues (like fractions of a cent).

## Task 5a

In the case of mismatches detected in question 4, which of the two values seems more reliable for reporting purposes?

    A: Wartosc_Sprzedazy from the sales report file

    B: the calculated value: Ilosc_Sprzedana * Cena_Jednostkowa_Transakcji

B: Ilosc_Sprzedana * Cena_Jednostkowa_Transakcji

## Task 5b

Justify the answer provided in the previous question.

In my opinion, Value B – the result of Ilosc_Sprzedana * Cena_Jednostkowa_Transakcji – is more reliable for reporting purposes, because it’s less prone to error as it’s based on a straightforward mathematical calculation.

Value A, on the other hand, may contain mistakes – for example, due to manual data entry, or it might include discounts that aren't visible in other columns.
With Value B, we’re relying on two separate numeric fields that are easier to verify and validate independently.